## 입력 데이터:
* 다운로드: https://github.com/hyerica-bdml/class-2020-datascience/tree/master/Week%208/input.zip
* 설명 - 영화DB사이트 (MovieLens)에서 수집된 데이터로서 u.data에는 사용자가 영화에 매긴 평점이 담겨있고 u.item에는 영화마다 제목, 카테고리 등의 정보가 담겨 있다.
#### 입력파일 1 - u.data
* 라인마다 사용자의 영화평점이 기록되어 있으며 4개의 속성 "user id \t item id \t rating \t timestamp"가 탭으로 구분되어 작성되어 있다. (즉, split("\t") 로 자를 수 있음)
* 모두 integer값이다.
* 한 사람이 여러개의 영화에 평점을 매길 수 있으므로 같은 user id로 시작하는 라인이 복수번 나타날수 있다.
* item id는 아래 u.item파일에 있는 영화 아이디 (movie id)를 가리킨다 (즉, foreign key)
* rating은 1에서 5까지 범위이다.

***힌트*** Data frame으로 읽는 방법: 
 ````python
 from pyspark.sql.types import *
 udataschema = StructType([
    StructField("uid", LongType(), False),
    StructField("mid", LongType(), False),
    StructField("rate",LongType(), False),
    StructField("timestamp", LongType(), False)
    ]) 
 udata = spark.read.format("csv").option("sep", "\t").schema(udataschema).load("u.data")
 ````

#### 입력파일 2 - u.item
* 라인마다 영화 정보가 기록되어 있으며 "movie id | movie title | release date | video release date |IMDb URL | unknown | Action | Adventure | Animation |Children's | Comedy | Crime | Documentary | Drama | Fantasy |Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |Thriller | War | Western" 의 속성 들이 파이프라인(|)으로 구분되어 작성되어 있다.
* 그러나 아래 문제를 보면 맨 앞의 두개 속성, movie id와 movie title 만 사용한다.
* movie id는 이 파일 안에서 한번만 나타난다 (즉, primary key)

***힌트*** Data frame으로 읽는 방법:
 ````python
 uitemschema = StructType([
    StructField("mid", LongType(), False),
    StructField("title",StringType(), False)
 ])
 uitem = spark.read.format("csv").option("sep", "|").schema(uitemschema).load("u.item")
 ````
## 문제 1 
* 5점을 받은 영화 중에서 알파벳 'T'로 시작하는 영화의 개수?
* 'T'는 대문자
* 주의: 5점을 받은 영화중 알파벳 'T'로 시작하는 영화를 중복해서 세지 않도록!
* 제한조건: 위 코드를 이용해 u.data와 u.item을 DataFrame으로 읽어들여 select및 join을 이용하여 계산하시오.
* 제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.
* 힌트: filter를 수행 시 filter조건을 표현할 때 어떤 column의 부분문자열을 뽑기 위해 pyspark.sql.functions.substring를 사용할 수 있음. 예를 들어 uitem이란 data frame의 title이란 column의 첫 글자는 substring(uitem["title"], 0, 1)으로 지정할 수 있음.

## 문제 2 
* 사용자들에게 1점도 받고 5점도 받은 적이 있는 영화의 제목의 길이 중 가장 긴 영화제목의 길이?
* 동일 영화가 어떤 사용자에게는 1점, 어떤 사용자에게는 5점을 받은 경우를 말함
* 힌트: 어떤 column의 길이의 max 를 select할 때 max 함수와 length함수를 사용.
* 예를 들어 uitem["title"]의 길이의 max는 select(max(length(uitem["title"]))) 로 계산 할 수 있음
* 제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.

In [26]:
import findspark
findspark.init()

# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my app").master("local").getOrCreate()

# get context from the session
sc = spark.sparkContext

In [27]:
from pyspark.sql.types import *

udataschema = StructType([
    StructField("uid", LongType(), False),
    StructField("mid", LongType(), False),
    StructField("rate",LongType(), False),
    StructField("timestamp", LongType(), False)
])
udata = spark.read.format("csv").option("sep", "\t").schema(udataschema).load("u.data")
print(udata.show(3))
udata.printSchema()

+---+---+----+---------+
|uid|mid|rate|timestamp|
+---+---+----+---------+
|196|242|   3|881250949|
|186|302|   3|891717742|
| 22|377|   1|878887116|
+---+---+----+---------+
only showing top 3 rows

None
root
 |-- uid: long (nullable = true)
 |-- mid: long (nullable = true)
 |-- rate: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [28]:
uitemschema = StructType([
    StructField("mid", LongType(), False),
    StructField("title",StringType(), False)
])
uitem = spark.read.format("csv").option("sep", "|").schema(uitemschema).load("u.item")
print(uitem.show(3))
uitem.printSchema()

+---+-----------------+
|mid|            title|
+---+-----------------+
|  1| Toy Story (1995)|
|  2| GoldenEye (1995)|
|  3|Four Rooms (1995)|
+---+-----------------+
only showing top 3 rows

None
root
 |-- mid: long (nullable = true)
 |-- title: string (nullable = true)



In [29]:
# join
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").show(3)

+---+--------------------+---+---+----+---------+
|mid|               title|uid|mid|rate|timestamp|
+---+--------------------+---+---+----+---------+
|242|        Kolya (1996)|196|242|   3|881250949|
|302|L.A. Confidential...|186|302|   3|891717742|
|377| Heavyweights (1994)| 22|377|   1|878887116|
+---+--------------------+---+---+----+---------+
only showing top 3 rows



In [30]:
from pyspark.sql.functions import *
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").filter( col('rate') == 5  ).show(3)

+----+--------------------+---+----+----+---------+
| mid|               title|uid| mid|rate|timestamp|
+----+--------------------+---+----+----+---------+
| 465|Jungle Book, The ...|253| 465|   5|891628467|
|1014|Romy and Michele'...|286|1014|   5|879781125|
| 222|Star Trek: First ...|200| 222|   5|876042340|
+----+--------------------+---+----+----+---------+
only showing top 3 rows



In [31]:
from pyspark.sql.functions import *
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").filter( col('title')[0:1] == 'T').show(3)

+---+--------------------+---+---+----+---------+
|mid|               title|uid|mid|rate|timestamp|
+---+--------------------+---+---+----+---------+
| 40|To Wong Foo, Than...|210| 40|   3|891035994|
|118|      Twister (1996)|291|118|   2|874833878|
|  1|    Toy Story (1995)|308|  1|   4|887736532|
+---+--------------------+---+---+----+---------+
only showing top 3 rows



In [32]:
from pyspark.sql.functions import *
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").filter( substring(uitem["title"], 0, 1) == 'T').show(3)

+---+--------------------+---+---+----+---------+
|mid|               title|uid|mid|rate|timestamp|
+---+--------------------+---+---+----+---------+
| 40|To Wong Foo, Than...|210| 40|   3|891035994|
|118|      Twister (1996)|291|118|   2|874833878|
|  1|    Toy Story (1995)|308|  1|   4|887736532|
+---+--------------------+---+---+----+---------+
only showing top 3 rows



In [10]:
from pyspark.sql.functions import *
uitem\
    .join(udata, uitem["mid"] == udata["mid"], "inner")\
    .filter(col('rate') == '5')\           # filter(udata["rate"] == 5)\
    .filter( col('title')[0:1] == 'T')\    # filter(substring(uitem["title"], 0, 1) == "T")
    .take(5)

[Row(mid=427, title='To Kill a Mockingbird (1962)', uid=60, mid=427, rate=5, timestamp=883326620),
 Row(mid=96, title='Terminator 2: Judgment Day (1991)', uid=42, mid=96, rate=5, timestamp=881107178),
 Row(mid=195, title='Terminator, The (1984)', uid=44, mid=195, rate=5, timestamp=878347874),
 Row(mid=195, title='Terminator, The (1984)', uid=72, mid=195, rate=5, timestamp=880037702),
 Row(mid=23, title='Taxi Driver (1976)', uid=59, mid=23, rate=5, timestamp=888205300)]

In [17]:
from pyspark.sql.functions import *
uitem\
    .join(udata, uitem["mid"] == udata["mid"], "inner")\
    .filter( col('rate') == '5')\
    .filter( col('title')[0:1] == 'T')\            # .filter( (col('rate') == 5) & (col('title')[0:1] == 'T') )\
    .select( udata['mid'] ).take(5)
#     .select( uitem['mid'] ).take(5) # 동일한 결과

[Row(mid=427), Row(mid=96), Row(mid=195), Row(mid=195), Row(mid=23)]

In [16]:
from pyspark.sql.functions import *
uitem\
    .join(udata, uitem["mid"] == udata["mid"], "inner")\
    .filter( col('rate') == '5')\
    .filter( col('title')[0:1] == 'T')\
    .select( uitem['mid'] )\
    .distinct().count()

75

In [19]:
uitem\
    .join(udata, uitem["mid"] == udata["mid"], "inner")\
    .filter( (col('rate') == 5) & (col('title')[0:1] == 'T') )\
    .distinct().count() # 전체 row비교하므로 같은 것이 많이 없다 ==> select ( uitem['mid']) 필요

1507

In [20]:
uitem.select("mid", "title")\
.join(udata.select("mid", "rate"), uitem["mid"] == udata["mid"], "inner")\
.filter( (col('rate') == 5) & (col('title')[0:1] == 'T') )\
.distinct().count()

75

## 문제2 : 1점도 받고 5점도 받은 영화의 제목의 길이중 가장 긴 영화제목의 길이는

In [21]:
from pyspark.sql.functions import *
uitem.select("mid", "title")\
    .join(udata.select("mid", "rate"), uitem["mid"] == udata["mid"], "inner")\
    .select("title", "rate")\
    .filter( col('rate') == (1 & 5) )\
    .select( max( length( col( "title") ) )).show() # 81

+------------------+
|max(length(title))|
+------------------+
|                81|
+------------------+



In [22]:
udata1 = udata.filter(udata["rate"] == 1).select("mid").distinct()
udata2 = udata.filter(udata["rate"] == 5).select("mid").distinct()
udata1\
    .join(udata2, "mid")\
    .join(uitem, udata1["mid"] == uitem["mid"])\
    .select(max(length(uitem["title"])))\
    .show()

+------------------+
|max(length(title))|
+------------------+
|                81|
+------------------+



In [23]:
sc.stop()